In [2]:
import pickle
import json

def read_pickle(file):
    with open(file, 'rb') as f:
        ret = pickle.load(f)
    return ret

def read_json(file):
    with open(file, 'r') as f:
        ret = [json.loads(line) for line in f]
    return ret

In [3]:
filepath = '../yelp_dataset/filtered/'
jsonpath = '../yelp_dataset/json/'

In [4]:
# reviews = read_pickle(filepath+'reviews.pickle')
users = read_pickle(filepath+'users.pickle')
businesses = read_pickle(filepath+'businesses.pickle')

In [5]:
busi_unfil = read_json(jsonpath+'yelp_academic_dataset_business.json')

In [6]:
user_unfil = read_json(jsonpath+'yelp_academic_dataset_user.json')
print(user_unfil[0].keys())

dict_keys(['funny', 'friends', 'user_id', 'name', 'useful', 'compliment_photos', 'compliment_cool', 'compliment_profile', 'compliment_list', 'review_count', 'average_stars', 'compliment_more', 'cool', 'fans', 'elite', 'compliment_cute', 'compliment_plain', 'compliment_note', 'compliment_hot', 'compliment_writer', 'yelping_since', 'compliment_funny'])


In [1]:
print(type(busi_unfil[0]))
print(busi_unfil[0].keys())
print(len(busi_unfil))
print(busi_unfil[0]['categories'])

NameError: name 'busi_unfil' is not defined

In [13]:
busi_filtered = []
for busi in busi_unfil:
    for business in businesses:
        if busi['business_id'] == business:
            busi_filtered.append(busi)
            break
user_filtered = []
for user in user_unfil:
    for u in users:
        if user['user_id'] == u:
            user_filtered.append(user)
            break

In [8]:
print(len(busi_filtered))
print(len(businesses))

12852
12852


In [9]:
uinds = [i for i in range(len(users))]
uid2ind = {user:ind for user, ind in zip(users, uinds)}
ind2uid = {ind:user for user, ind in zip(users, uinds)}

b_inds = [i for i in range(len(businesses))]
bid2ind = {business:ind for business, ind in zip(businesses, b_inds)}
ind2bid = {ind:business for business, ind in zip(businesses, b_inds)}

ct_id2ind = {busi['city']:ind for busi, ind in zip(busi_filtered, b_inds)}
ind2ct_id = {ind:busi['city'] for busi, ind in zip(busi_filtered, b_inds)}

ca_id2ind = {busi['categories']:ind for busi, ind in zip(busi_filtered, b_inds)}
ind2ca_id = {ind:busi['categories'] for busi, ind in zip(busi_filtered, b_inds)}

In [11]:
import numpy as np
Y = np.zeros((len(users)*len(businesses), 3))
print(Y)

MemoryError: Unable to allocate array with shape (162474984, 3) and data type float64

In [12]:
# get adjs
def get_adj_matrix(uid2ind, bid2ind, city_id2ind, cat_id2ind, users, businesses):
    """
    metapaths: UB, UUB, UBUB, UBCaB, UBCiB
    """
    tot_users = len(uid2ind)  # tot for total
    tot_business = len(bid2ind)
    tot_city = len(city_id2ind)
    tot_category = len(cat_id2ind)
    #relation U-U
    adj_UU = np.zeros([tot_users, tot_users])
    adj_UB = np.zeros([tot_users, tot_business])
    adj_BCa = np.zeros([tot_business, tot_category])
    adj_BCi = np.zeros([tot_business, tot_city])
    for user in users:
        if user['user_id'] not in uid2ind:
            continue
        user_id = uid2ind[user['user_id']]
        for friend in user['friends'].split(','):
            friend = friend.strip()
            if friend in uid2ind:
                friend_id = uid2ind[friend]
                adj_UU[user_id][friend_id] = 1
                adj_UU[friend_id][user_id] = 1
    #relation U-P-B
    for review in pos_reviews:
        user_id = review[0]
        business_id = review[1]
        adj_UPB[user_id][business_id] = 1
    #relation B_Ca B_Ci
    for business in businesses:
        if business['business_id'] not in bid2ind:
            continue
        business_id = bid2ind[business['business_id']]
        city_id = city_id2ind[business['city']]
        adj_BCi[business_id][city_id] = 1
        adj_CiB[city_id][business_id] = 1
        # more than one category for a business
        for category in business['categories'].split(','):
            category = category.strip()
            category_id = cat_id2ind[category]
            adj_BCa[business_id][category_id] = 1
            adj_CaB[category_id][business_id] = 1

    #metapath
    adj_UUB = adj_UU.dot(adj_UB)

    adj_UBU = adj_UB.dot(adj_UB.T)

    adj_UBUB = adj_UBU.dot(adj_UB)

    adj_UBCa = adj_UB.dot(adj_BCa)
    adj_UBCaB = adj_UBCa.dot(adj_BCa.T)

    adj_UBCi = adj_UB.dot(adj_BCi)
    adj_UBCiB = adj_UBCi.dot(adj_BCi.T)

#     adj_UCaB = adj_UCa.dot(adj_CaB)
    
#     adj_UCiB = adj_UCi.dot(adj_CiB)
    
    return adj_UB, adj_UUB, adj_UBUB, adj_UBCaB, adj_UBCiB


In [1]:
# get adj matrices
adj_UB, adj_UUB, adj_UBUB, adj_UBCaB, adj_UBCiB \
    = get_adj_matrix(uid2ind, bid2ind, ct_id2ind, ca_id2ind, user_filtered, busi_filtered)

NameError: name 'get_adj_matrix' is not defined

In [15]:
# partition dataset

{1, 2, 3, 4, 5}
